# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Data Files to study
weather_metadata = "my_weather_data.csv"

# Read the data and study results
weather_data = pd.read_csv(weather_metadata)

# #Combine data into single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,73.83,81,100,9.93,PF,1650823200
1,1,Busselton,-33.6500,115.3333,65.52,51,100,12.39,AU,1650823200
2,2,Saint George,37.1041,-113.5841,61.05,24,0,4.81,US,1650823200
3,3,Thompson,55.7435,-97.8558,20.73,62,60,17.92,CA,1650823200
4,4,Acapulco de Juárez,16.8634,-99.8901,86.38,61,9,9.84,MX,1650823200


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and "Lng" into locations
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

hum_rate = weatherdf["Humidity"].astype(float)

In [6]:
# Create a poverty Heatmap Layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
132,132,Lompoc,34.6391,-120.4579,72.95,57,0,7.70,US,1650823200
134,134,Roebourne,-20.7833,117.1333,79.56,61,0,7.23,AU,1650823200
170,170,Etchoropo,26.7667,-109.6667,79.86,44,0,9.82,MX,1650823200
174,174,Cabo San Lucas,22.8909,-109.9124,74.62,72,0,6.40,MX,1650823200
187,187,Rancho Mirage,33.7397,-116.4128,78.75,11,0,7.74,US,1650823200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
132,Lompoc,US,34.6391,-120.4579
134,Roebourne,AU,-20.7833,117.1333
170,Etchoropo,MX,26.7667,-109.6667
174,Cabo San Lucas,MX,22.8909,-109.9124
187,Rancho Mirage,US,33.7397,-116.4128


In [9]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 132: Lompoc.
Closest hotel to Lompoc at 34.6391 , -120.4579 is Red Roof Inn Lompoc.
------------
Retrieving Results for Index 134: Roebourne.
Closest hotel to Roebourne at -20.7833 , 117.1333 is Roebourne Police.
------------
Retrieving Results for Index 170: Etchoropo.
Closest hotel to Etchoropo at 26.7667 , -109.6667 is Tienda.
------------
Retrieving Results for Index 174: Cabo San Lucas.
Closest hotel to Cabo San Lucas at 22.8909 , -109.9124 is Hotel Santa Fe Cabo San Lucas.
------------
Retrieving Results for Index 187: Rancho Mirage.
Closest hotel to Rancho Mirage at 33.7397 , -116.4128 is Shadow Mountain Resort & Club.
------------
Retrieving Results for Index 396: Guerrero Negro.
Closest hotel to Guerrero Negro at 27.9769 , -114.0611 is Sedena 40 Z.M.
------------
Retrieving Results for Index 456: Tura.
Closest hotel to Tura at 25.5198 , 90.2201 is State Bank of India - Evening Branch.
------------


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat maps
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>